In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, udf, datediff, round, count, desc, isnan
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import row_number, desc, col, lit
from pyspark.sql.functions import col, when
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import LinearRegression

In [2]:
g_spark = SparkSession.builder \
   .appName("Prabhu_ml") \
   .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/12 02:26:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/12 02:26:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/12 02:26:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/06/12 02:26:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/06/12 02:26:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/06/12 02:26:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
24/06/12 02:26:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
24/06/12 02:26:10 WARN util.Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 

In [31]:
#df = spark.read.parquet('hdfs://localhost:9000/ingest/pbldb/gsa_lam_table')

In [3]:
data = g_spark.read.format("parquet") \
                .option("header", "true") \
                .option("inferSchema", "true") \
                .load("hdfs://localhost:9000/ingest/ml_dataset/gsa_lam_demo_int_table")

In [6]:
new_dataframe = data.select(*(col(c).cast("float").alias(c) for c in data.columns))

In [7]:
new_dataframe.printSchema()

root
 |-- acid: float (nullable = true)
 |-- entity_cre_flg: float (nullable = true)
 |-- acct_prefix: float (nullable = true)
 |-- acct_num: float (nullable = true)
 |-- bacid: float (nullable = true)
 |-- foracid: float (nullable = true)
 |-- acct_name: float (nullable = true)
 |-- acct_short_name: float (nullable = true)
 |-- cust_id: float (nullable = true)
 |-- emp_id: float (nullable = true)
 |-- cif_id: float (nullable = true)
 |-- schm_code: float (nullable = true)
 |-- schm_desc: float (nullable = true)
 |-- acct_opn_date: float (nullable = true)
 |-- acct_cls_date: float (nullable = true)
 |-- clr_bal_amt: float (nullable = true)
 |-- acct_cls_flg: float (nullable = true)
 |-- tot_mod_times: float (nullable = true)
 |-- ledg_num: float (nullable = true)
 |-- un_clr_bal_amt: float (nullable = true)
 |-- inter_sol_access_flg: float (nullable = true)
 |-- purge_allowed_flg: float (nullable = true)
 |-- purge_text: float (nullable = true)
 |-- acct_mgr_user_id: float (nullable = 

In [8]:
#all columns of data
for c in new_dataframe.columns:
    print(c)

acid
entity_cre_flg
acct_prefix
acct_num
bacid
foracid
acct_name
acct_short_name
cust_id
emp_id
cif_id
schm_code
schm_desc
acct_opn_date
acct_cls_date
clr_bal_amt
acct_cls_flg
tot_mod_times
ledg_num
un_clr_bal_amt
inter_sol_access_flg
purge_allowed_flg
purge_text
acct_mgr_user_id
schm_type
partitioned_flg
partitioned_type
sol_id
del_flg
sol_desc
city_code
state_code
bank_code
extn_cntr_code
br_code
accountid
preferred_mobile_alert_no
preferred_mobile_alert_type
submitforkyc
kyc_reviewdate
kyc_date
riskrating
salutation
gender
cust_dob
education
occupation
cust_language
demographicid
marital_status
employment_status
income_nature
annual_salary_income
annual_rental_income
annual_stock_bond_income
annual_others_income
annual_total_income
salotherincome2
selfempaverageannualturnover
totalhouseholdincm
employersname
sourceofincome
accrued_upto_date_cr
accrued_upto_date_dr
last_accrual_run_date_cr
last_accrual_run_date_dr
booked_upto_date_cr
booked_upto_date_dr
last_book_run_date_cr
last_boo

In [9]:
new_dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in new_dataframe.columns]).show()

24/06/12 03:06:06 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+--------------+-----------+--------+------+-------+---------+---------------+-------+------+------+---------+---------+-------------+-------------+-----------+------------+-------------+--------+--------------+--------------------+-----------------+----------+----------------+---------+---------------+----------------+------+-------+--------+---------+----------+---------+--------------+-------+---------+-------------------------+---------------------------+------------+--------------+--------+----------+----------+------+--------+---------+----------+-------------+-------------+--------------+-----------------+-------------+--------------------+--------------------+------------------------+--------------------+-------------------+---------------+----------------------------+------------------+-------------+--------------+--------------------+--------------------+------------------------+------------------------+-------------------+-------------------+---------------------+----

In [10]:
new_data= data.dropDuplicates(["acid"])

In [11]:
columns_to_select = [
    "schm_desc", "kyc_date", "riskrating", "gender","sol_id","sol_desc" ,"occupation",
    "employment_status", "annual_salary_income", "annual_total_income",
    "totalhouseholdincm", "sourceofincome", "interest_rate", "dis_amt"
]

# Select the specified columns
df_selected = new_data.select(*columns_to_select)

In [38]:
selected_df = df_selected.filter(df.sol_id.isin([999, 1, 111]))
selected_df.show()

+--------------------+--------------------+----------+------+------+--------------------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+------------+
|           schm_desc|            kyc_date|riskrating|gender|sol_id|            sol_desc|occupation|employment_status|annual_salary_income|annual_total_income|totalhouseholdincm|sourceofincome|interest_rate|     dis_amt|
+--------------------+--------------------+----------+------+------+--------------------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+------------+
|STAFF HOUSING LOA...|2020-05-29 00:00:...|    MEDIUM|     M|   999|HEAD OFF, BABARMAHAL|     OTHER|            Other|                 1.0|                1.0|              null|          null|          0.0|2127000.0000|
|STAFF HOUSING LOA...|2020-05-29 00:00:...|    MEDIUM|     M|   999|HEAD OFF, BABARMAHAL|     OTHER|         Salarie

In [39]:
rows_count = selected_df.count()
rows_count

4701

In [42]:
occupation_counts = selected_df.groupBy("occupation").count()


In [43]:
# Get the top 10 occupations
top_occupations = occupation_counts.orderBy(col("count").desc()).limit(10).select("occupation")

# Collect the top 10 occupations as a list
top_occupation_list = [row['occupation'] for row in top_occupations.collect()]



TypeError: 'str' object is not callable

In [17]:
df_modified = df_selected.withColumn(
    "occupation",
    when(col("occupation").isin(top_occupation_list), col("occupation")).otherwise("others")
)

In [18]:
typs_counts = df_modified.groupBy("schm_desc").count()

# Get the top 10 occupations
top_type = typs_counts.orderBy(col("count").desc()).limit(10).select("schm_desc")

# Collect the top 10 occupations as a list
top_type_list = [row['schm_desc'] for row in top_type.collect()]

# Replace occupations not in the top 10 with "others"
df_modified = df_modified.withColumn(
    "schm_desc",
    when(col("schm_desc").isin(top_type_list), col("schm_desc")).otherwise("others")
)

In [19]:
df = df_modified.filter(~when(df_modified.annual_salary_income == 1, True).otherwise(False))

In [20]:
df = df.filter(~df.annual_salary_income.rlike(r'[0-9\.]+E[+-]?[0-9]+'))
df = df.filter(~when(df.annual_total_income == 1, True).otherwise(False))
df = df.filter(~when(df.totalhouseholdincm == 1, True).otherwise(False))
df = df.filter(~df.annual_total_income.rlike(r'[0-9\.]+E[+-]?[0-9]+'))
df = df.filter(~df.totalhouseholdincm.rlike(r'[0-9\.]+E[+-]?[0-9]+'))

In [21]:
df = df.withColumn('kyc_date', when(col('kyc_date').isNull(), 0).otherwise(1))
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+--------+----------+------+------+--------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+-------+
|schm_desc|kyc_date|riskrating|gender|sol_id|sol_desc|occupation|employment_status|annual_salary_income|annual_total_income|totalhouseholdincm|sourceofincome|interest_rate|dis_amt|
+---------+--------+----------+------+------+--------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+-------+
|        0|       0|         0|     0|    42|      42|         0|                0|                   0|                  0|                 0|         14696|            0|      0|
+---------+--------+----------+------+------+--------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+-------+



In [22]:
df.show()

+--------------------+--------+----------+------+------+--------------------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+------------+
|           schm_desc|kyc_date|riskrating|gender|sol_id|            sol_desc|occupation|employment_status|annual_salary_income|annual_total_income|totalhouseholdincm|sourceofincome|interest_rate|     dis_amt|
+--------------------+--------+----------+------+------+--------------------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+------------+
|              others|       0|    MEDIUM|     M|   050|       PARASI BRANCH|     SALPN|         Salaried|            219288.0|           569288.0|          569288.0|          null|       -13.35| 263816.6200|
|              others|       1|    MEDIUM|     M|   050|       PARASI BRANCH|     NARMY|            Other|            500000.0|          1000000.0|         1000000.

In [23]:
df = df.withColumn("dis_amt", col("dis_amt").cast("double"))

In [24]:
unique_schm_desc = df.select('schm_desc').distinct()

In [25]:
df.show()

+--------------------+--------+----------+------+------+--------------------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+----------+
|           schm_desc|kyc_date|riskrating|gender|sol_id|            sol_desc|occupation|employment_status|annual_salary_income|annual_total_income|totalhouseholdincm|sourceofincome|interest_rate|   dis_amt|
+--------------------+--------+----------+------+------+--------------------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+----------+
|              others|       0|    MEDIUM|     M|   050|       PARASI BRANCH|     SALPN|         Salaried|            219288.0|           569288.0|          569288.0|          null|       -13.35| 263816.62|
|              others|       1|    MEDIUM|     M|   050|       PARASI BRANCH|     NARMY|            Other|            500000.0|          1000000.0|         1000000.0|      

In [26]:
from pyspark.ml.feature import StringIndexer

columns = ['schm_desc', 'riskrating', 'gender', 'employment_status']

for col in columns:
    indexer = StringIndexer(inputCol=col, outputCol=col+"_indexed")
    df = indexer.fit(df).transform(df)

In [27]:
features = df.drop('schm_desc', 'riskrating', 'gender', 'employment_status','sourceofincome')

In [28]:
features.show()

+--------+------+--------------------+----------+--------------------+-------------------+------------------+-------------+----------+-----------------+------------------+--------------+-------------------------+
|kyc_date|sol_id|            sol_desc|occupation|annual_salary_income|annual_total_income|totalhouseholdincm|interest_rate|   dis_amt|schm_desc_indexed|riskrating_indexed|gender_indexed|employment_status_indexed|
+--------+------+--------------------+----------+--------------------+-------------------+------------------+-------------+----------+-----------------+------------------+--------------+-------------------------+
|       0|   050|       PARASI BRANCH|     SALPN|            219288.0|           569288.0|          569288.0|       -13.35| 263816.62|              4.0|               0.0|           0.0|                      2.0|
|       1|   050|       PARASI BRANCH|     NARMY|            500000.0|          1000000.0|         1000000.0|        -12.0| 226996.01|              

In [29]:
assembler = VectorAssembler(inputCols=[
 'kyc_date',
 'annual_salary_income',
 'annual_total_income',
 'totalhouseholdincm',
 'schm_desc_indexed',
 'interest_rate',
 'dis_amt',
 'gender_indexed',
 'employment_status_indexed'],
 outputCol='Independant_Features')

output  = assembler.transform(features)

In [ ]:
final_df = output.select("Independant_Features","riskrating_indexed")

training, test = final_df.randomSplit([0.7, 0.3])
random_forest_reg = RandomForestRegressor(featuresCol="Independant_Features",labelCol="riskrating_indexed",maxBins=200)

model = random_forest_reg.fit(training)

predictions_1 = model.transform(training) # checking the accuracy of the traning dataset

prediction_2 = model.transform(test) # checking the accuracy of the test datase
evaluator = RegressionEvaluator(labelCol="riskrating_indexed",\
                                predictionCol="prediction", metricName="r2")
print("R Squared (R2) on traning data =", evaluator.evaluate(predictions_1))


evaluator = RegressionEvaluator(labelCol="riskrating_indexed",\
                                predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(prediction_2))


R Squared (R2) on traning data = 0.10764928770644289
R Squared (R2) on test data = 0.10428946154805463


In [31]:
evaluator = RegressionEvaluator(labelCol="riskrating_indexed",\
                                predictionCol="prediction", metricName="r2")
print("R Squared (R2) on traning data =", evaluator.evaluate(predictions_1))


evaluator = RegressionEvaluator(labelCol="riskrating_indexed",\
                                predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(prediction_2))

R Squared (R2) on traning data = 0.10764928770644289
R Squared (R2) on test data = 0.10428946154805463


In [125]:
df = df.na.fill('others', subset=['Department']) 

+-------+---------+-------------------+----------+------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------------+------------------+------------------+-------------------+------------------+-------------------------+
|summary|schm_desc|           kyc_date|riskrating|gender|occupation|employment_status|annual_salary_income|annual_total_income|totalhouseholdincm|sourceofincome|      interest_rate|           dis_amt| schm_desc_indexed| riskrating_indexed|    gender_indexed|employment_status_indexed|
+-------+---------+-------------------+----------+------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------------+------------------+------------------+-------------------+------------------+-------------------------+
|  count|    30243|              30243|     30243| 30243|     30243|            30243|               30243|              30243|             30243

In [128]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---------+--------+----------+------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+-------+-----------------+------------------+--------------+-------------------------+
|schm_desc|kyc_date|riskrating|gender|occupation|employment_status|annual_salary_income|annual_total_income|totalhouseholdincm|sourceofincome|interest_rate|dis_amt|schm_desc_indexed|riskrating_indexed|gender_indexed|employment_status_indexed|
+---------+--------+----------+------+----------+-----------------+--------------------+-------------------+------------------+--------------+-------------+-------+-----------------+------------------+--------------+-------------------------+
|        0|       0|         0|     0|         0|                0|                   0|                  0|                 0|             0|            0|      0|                0|                 0|             0|                        0|
+---------+--------+--------

In [ ]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Define the Gradient Boosting Regressor
gbt = GBTRegressor(labelCol="riskrating_indexed", featuresCol="Independant_Features")

# Hyperparameter tuning
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [5, 10]) \
    .addGrid(gbt.maxIter, [50, 100]) \
    .build()

crossval = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="riskrating_indexed", predictionCol="prediction", metricName="rmse"),
                          numFolds=3)

cvModel = crossval.fit(training)
bestModel = cvModel.bestModel


24/06/11 04:57:09 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/11 04:57:09 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
24/06/11 05:03:48 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1006.0 KiB
24/06/11 05:03:50 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1008.8 KiB
24/06/11 05:03:51 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1009.3 KiB
24/06/11 05:03:51 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1010.0 KiB
24/06/11 05:03:52 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1011.1 KiB
24/06/11 05:03:52 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1013.5 KiB
24/06/11 05:03:53 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1018.9 KiB
24/06/11 05:03:53 WARN scheduler.DAGScheduler: 